This notebook provides a convenient wrapper for bag_reader.  
It looks for all bagfiles in a specified directory and converts to pickle/matfile for further processing.

In [1]:
import glob
import pickle
import scipy.io as sio
from bag_reader import *

/media/govvijay/data/carla/carla_0_9_6/PythonAPI/analysis/transformations.py:1926: UserWarning: No module named '_transformations'
  warnings.warn(str(err))


In [2]:
bagfiles_to_process = glob.glob('../examples/bags/*.bag')

print('Found the following bagfiles: %s' % bagfiles_to_process)

Found the following bagfiles: ['../examples/bags/parking_p2_t2_e0_2019-11-04-16-36-50.bag', '../examples/bags/parking_p1_t1_e2_2019-11-04-14-35-28.bag']


### Topics processed by bag_reader.py

* [x] /carla/actor_list
* [x] /carla/hero/collision
* [x] /carla/hero/gnss/front/fix
* [x] /carla/hero/objects
* [x] /carla/hero/odometry
* [x] /carla/hero/vehicle_info
* [x] /carla/hero/vehicle_status
* [x] /carla/odometry
* [x] /carla/status
* [x] /carla/world_info
* [x] /intention
* [ ] <span style="color:blue"> /carla/objects </span>
* [ ] <span style="color:blue"> /rosout </span>
* [ ] <span style="color:blue"> /rosout_agg </span>
* [ ] <span style="color:blue"> /tf </span>
* [ ] <span style="color:blue"> /carla/camera/rgb/front/camera_info </span>
* [ ] <span style="color:blue"> /carla/camera/rgb/front/image_color </span>
* [ ] <span style="color:blue"> /carla/marker </span>
* [ ] <span style="color:blue"> /clock </span>

### Additional Notes
<span style="color:blue"> We do not plan to process blue topics (or will do it separately for images). /carla/objects contains the ego vehicle as well, so excluded. </span>

<span style="color:red;font-weight:bold"> Outlier detection is not perfect - only the trajectory from ego odometry is impacted if a jump is noticed. </span>

<span style="color:red;font-weight:bold"> TODO: extend to other time-varying fields. </span>


In [3]:
for bagname in bagfiles_to_process:
    print('Processing ', bagname)
    result_dict = process_bag(bagname)
    
    fname = bagname.split('.bag')[0]
    assert('parking' in fname)
    
    '''
    # Save to file (matfile, needs work)
    fname += '.mat'
    print('Saving ', fname)
    sio.savemat(fname, result_dict)
    '''
    
    fname += '.pkl'
    pickle.dump(result_dict, open(fname, 'wb'))
    
print('Finished', fname)

Processing  ../examples/bags/parking_p2_t2_e0_2019-11-04-16-36-50.bag
Outlier removed at time 1255.524, last entry at time 1255.690
Processing  ../examples/bags/parking_p1_t1_e2_2019-11-04-14-35-28.bag
Outlier removed at time 282.791, last entry at time 283.658
Finished ../examples/bags/parking_p1_t1_e2_2019-11-04-14-35-28.pkl
